In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import pearsonr
random.seed(1234)
np.random.seed(1234)
np.random.rand()

0.1915194503788923

In [28]:
class Buyer:
    # This class defines a buyer. A buyer has a reservation value above
    # which he will not trade. Since each buyer can only trade once, we
    # record whether a buyer traded or not. A buyer generates a bid price
    # (which varies depending if the simulation runs with or without
    # constraint)
    
    def __init__(self, quantity):
        self.Value = []
        self.Traded = 0
        self.Quantity = quantity
    
    # Generate a bid offer
    def formBidPrice(self):
        potentialBid = self.Value[self.Traded] - (np.random.rand() * (self.Value[self.Traded] - 0))
            
        return potentialBid

In [29]:
class Seller:
    # This class defines a seller. A seller has a reservation cost below
    # which he will not trade. Since each seller can only trade once, we
    # record whether a seller traded or not. A seller generates a bid price
    # (which varies depending if the simulation runs with or without
    # constraint)
    
    def __init__(self, quantity):
        self.Cost = []
        self.Traded = 0
        self.Quantity = quantity
    
    # Generate ask offer
    def formAskPrice(self):
        potentialAsk = self.Cost[self.Traded] + ((np.random.rand() * np.absolute(300 - self.Cost[self.Traded])))
        
        return potentialAsk

In [30]:
def initializeBook(maxCost):
    # Initialize order book variables. Record the best standing bid and ask,
    # the ID of the best bidders (index number in buyer/seller vectors), 
    # transaction price (initialized to 0) and logical variables to determine
    # if there currently is a standing bid or ask. Initially there are no best 
    # bidders, so index of each is 0. The order book vector also records the
    # added value of a given trade (initialized to 0)

    # First index is the best bid (initialized to 0)
    # Second index is the best bid ID (initialized to 0, nobody)
    # Third index is the logical variable for current bid (1 for true, initialized to 0)
    # Fourth index is the best ask (initialized to the max cost for sellers)
    # Fifth index is the best ask ID (initialized to 0, nobody)
    # Sixth index is the logical variable for current ask (1 for true, initialized to 0).
    # Seventh index is the transaction price (initialized to 0)  
    
    orderBookValues = [0 ,0 , 0, maxCost + 1,0, 0, 0, 0] 
    return orderBookValues

In [31]:
def doTrade(buyers, sellers, bookValues, numberTraders, Quantity):
    # This function operates the double auction order book. The function
    # determines with 50% probability whether the next trader to submit an
    # offer is a buyer or a seller. Then, it compares the new bid/ask and
    # executes a trade if it satisfies the counterposing best standing offer,
    # or it takes the place as best offer of its kind if it is better than the
    # currently standing one (for example, if a new bid is submitted and it is
    # higher than the best ask, a trade will occur, but if it's not higher than
    # the best ask but higher than the best standing bid it will take its
    # place. A similar process takes place if a seller submits an ask). The
    # function returns an updated state of the order book to update simulation.
    
    # Initialize return vector to current book values
    updatedValues = bookValues
    

    # Randomly choose between a buyer and a seller (belor 0.5 we select a
    # buyer, otherwise we select a seller).
    traderDeterminer = np.random.rand()
    # Process as buyer
    if traderDeterminer <0.5:
        
        # Initialize a new buyer ID (at first 0)
        newBuyer = -1
    
        #Choose a random buyer that has yet not traded. Lock selection of a
        # buyer in a while loop until it chooses a buyer that hasn't traded
        while newBuyer == -1:
            # Choose a random index in the buyers vector to select a buyer
            randomIndex = np.random.randint(0, numberTraders)
            if buyers[randomIndex].Traded != Quantity:
                newBuyer = randomIndex
        
        
        # Make buyer generate a random bid based on its reservation price and
        # simulation constraint
        newBid = buyers[newBuyer].formBidPrice()
        
        # Do a trade if there is currently a standing ask that the bid can
        # trade with. Check the logical variable and the value of the best
        # standing ask
        if (updatedValues[5] == 1) & (newBid > updatedValues[3]):
            # Set the transaction price to the best ask value
            updatedValues[6] = updatedValues[3]
            # Record surplus added by the trade
            # updatedValues[7] = buyers[newBuyer].Value - sellers[updatedValues[4]].Cost
            # Update ID of buyer
            updatedValues[1] = newBuyer

        
        # If the there is no trade, set the bid as best bid if it is higher
        # than the currently standing best bid, even if it doesn't satisfy the
        # ask or if there currently is no ask
        else:
            if newBid > updatedValues[0]:
                # Set new bid as best bid, and update the ID of bidder
                updatedValues[0]= newBid
                updatedValues[1] = newBuyer
                # Set logical variable for standing bid to true
                updatedValues[2] = 1
                # If new bid is lower than best bid, do nothing
    # Process a seller
    else:
        # Initialize a new seller ID (at first 0)
        newSeller = -1
        # Choose a random seller that has yet not traded. Lock selection of a
        # seller in a while loop until it chooses a seller that hasn't traded
        while newSeller == -1:
            # Choose a random index in the sellers vector to select a seller
            randomIndex = np.random.randint(0, numberTraders)
            if sellers[randomIndex].Traded != Quantity:
                newSeller = randomIndex
        
        # Make seller generate a random ask based on its reservation cost and
        # simulation constraint
        newAsk = sellers[newSeller].formAskPrice()
        
        # Do a trade if there is currently a standing bid that the ask can
        # trade with. Check the logical variable and the value of the best
        # standing bid
        if (updatedValues[2] == 1) & (updatedValues[0] > newAsk):
            # Set the transaction price to the best bid value
            updatedValues[6] = updatedValues[0]
            # Record surplus added by the trade
            #updatedValues[7] = buyers[updatedValues[1]].Value - sellers[newSeller].Cost
            # Record ID of seller
            updatedValues[4] = newSeller
        # If the there is no trade, set the ask as best ask if it is lower
        # than the currently standing best ask, even if it doesn't satisfy the
        # bid or if there currently is no bid
        else:
            if newAsk < updatedValues[3]:
                # Set new ask as best ask, and update the ID of bidder
                updatedValues[3] = newAsk
                updatedValues[4] = newSeller
                # Set logical variable for standing ask to true
                updatedValues[5] = 1
                # If new ask is higher than best ask, do nothing
       
    return updatedValues

In [32]:
def Average(lst): 
    return sum(lst)/len(lst)

def Rank(vector):
    a={}
    rank=1
    for num in sorted(vector):
        if num not in a:
            a[num]=rank
            rank=rank+1
    return[a[i] for i in vector]

In [7]:
# market 2
numberTraders = 30
maxCost = 120  
Quantity = 6

# Run simulation with 200 iteration
iterations = 2000

# Create vector holding all the values and costs
Values = [[150, 141, 132, 129, 120, 111],
          [147, 138, 129, 126, 117, 108],
          [145, 136, 127, 124, 115, 106],
          [150, 141, 138, 129, 120, 108],
          [149, 140, 137, 128, 119, 107],
          [143, 140, 131, 122, 110, 101],
          [142, 139, 130, 121, 109, 100],
          [148, 139, 130, 127, 118, 109],
          [149, 140, 131, 128, 119, 110],
          [148, 139, 136, 127, 118, 106],
          [142, 133, 130, 121, 112, 100],
          [150, 141, 132, 120, 111, 102],
          [149, 140, 131, 119, 110, 101],
          [148, 139, 130, 118, 109, 100],
          [150, 147, 138, 129, 117, 108],
          [149, 146, 137, 128, 116, 107],
          [146, 137, 128, 125, 116, 107],
          [143, 134, 131, 122, 113, 101],
          [142, 133, 124, 121, 112, 103],
          [148, 145, 136, 127, 115, 106],
          [147, 144, 135, 126, 114, 105],
          [146, 143, 134, 125, 113, 104],
          [145, 142, 133, 124, 112, 103],
          [144, 141, 132, 123, 111, 102],
          [147, 138, 135, 126, 117, 105],
          [144, 135, 132, 123, 114, 102],
          [145, 136, 133, 124, 115, 103],
          [146, 137, 134, 125, 116, 104],
          [144, 135, 126, 123, 114, 105],
          [143, 134, 125, 122, 113, 104]]
Costs = [[56, 67, 78, 86, 97, 108],
         [55, 66, 77, 85, 96, 107],
         [60, 71, 79, 90, 101, 120],
         [52, 63, 74, 82, 93, 104],
         [54, 65, 76, 84, 95, 106],
         [51, 62, 70, 81, 92, 111],
         [52, 63, 71, 82, 93, 112],
         [60, 68, 79, 90, 109, 120],
         [50, 61, 69, 80, 91, 110],
         [51, 62, 73, 81, 92, 103],
         [50, 61, 72, 80, 91, 102],
         [53, 64, 72, 83, 94, 113],
         [59, 70, 78, 89, 100, 119],
         [50, 58, 69, 80, 99, 110],
         [51, 59, 70, 81, 100, 111],
         [52, 60, 71, 82, 101, 112],
         [53, 61, 72, 83, 102, 113],
         [54, 62, 73, 84, 103, 114],
         [58, 69, 77, 88, 99, 118],
         [57, 68, 79, 87, 98, 109],
         [55, 63, 74, 85, 104, 115],
         [56, 64, 75, 86, 105, 116],
         [57, 65, 76, 87, 106, 117],
         [58, 66, 77, 88, 107, 118],
         [59, 67, 78, 89, 108, 119],
         [54, 65, 73, 84, 95, 114],
         [57, 68, 76, 87, 98, 117],
         [56, 67, 75, 86, 97, 116],
         [55, 66, 74, 85, 96, 115],
         [53, 64, 75, 83, 94, 105]]


# Create vector holding autocorreltion of price change
# and correlation of transaction order and costs/values
auto_corr = []
trx_order_seller = []
trx_order_buyer = []
q = []
p = []
hist = []

# Create vector holding all buyers
buyers = []
for i in range(numberTraders):
    buyers.append(Buyer(Quantity))

# Create vector holding all sellers
sellers = []
for i in range(numberTraders):
    sellers.append(Seller(Quantity))

print("Please wait......")
for i in range(10000):
    # Initialize all buyers to not traded state and give them a reservation
    # price (random variable bounded above)
    valueVec = np.asarray(Values)

    for i in range(numberTraders):
        valueVec[i].sort()
        valueVec[i] = valueVec[i][::-1]
        buyers[i].Value = valueVec[i]
        buyers[i].Traded = 0
    
    # Initialize all sellers to not traded state and give them a reservation
    # cost (random variable bounded below)
    costVec = np.asarray(Costs)

    for i in range(numberTraders):
        costVec[i].sort()
        sellers[i].Cost = costVec[i]
        sellers[i].Traded = 0


    # Initialize the order book vector. Please see function description for the
    # values held in each index   
    orderBookValues = initializeBook(maxCost)

    # Initialize vector with transaction prices, values, and costs (update as iterations execute).
    transactionPrices = []

    # Initial quantity traded is 0
    #quantity = 0
    tradedValues = []
    tradedCosts  = []
    h = []

    for i in range(iterations):
        # Stop the loop if all buyers and sellers have already traded all their goods.
        count = 0
        for i in range(numberTraders):
            if buyers[i].Traded == Quantity:
                count += 1
        if count == numberTraders:
            break
        # Attempt a trade or a new bid/ask (report update if trade occurs). 
        orderBookValues = doTrade(buyers, sellers, orderBookValues, numberTraders, Quantity)
    
        # Record transaction price, update surplus and quantity, mark traders 
        # to record that they have traded, and reinitialize the order book
        # if a trade occured
        if orderBookValues[6] > 0:
            transactionPrices.append(orderBookValues[6])
            #surplus = surplus + orderBookValues[7]
            tradedValues.append(buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded])
            tradedCosts.append(sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded])
            h.append([sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded], 
                      buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded], 
                      orderBookValues[6]])
            buyers[orderBookValues[1]].Traded += 1
            sellers[orderBookValues[4]].Traded += 1
            orderBookValues = initializeBook(maxCost)
            
        
    # gen transaction order and price change
    order = []
    for i in range(1, len(transactionPrices) + 1):
        order.append(i)

    all_priceChange = []
    for i in range(1, len(transactionPrices)):
        all_priceChange.append(transactionPrices[i] - transactionPrices[i - 1])

    priceChange = []
    lag_priceChange = []
    for i in range(1, len(all_priceChange)):
        priceChange.append(all_priceChange[i])
        lag_priceChange.append(all_priceChange[i-1])


    # Calculate correlation
    price_auto_corr, _ = pearsonr(priceChange, lag_priceChange)
    seller_corr, _ = pearsonr(Rank(tradedCosts), order)
    buyer_corr, _ = pearsonr(Rank(tradedValues), order)

    auto_corr.append(price_auto_corr)
    trx_order_seller.append(seller_corr)
    trx_order_buyer.append(buyer_corr)
    q.append(len(order))
    hist.append(h)

print("The simulation is done")

print("average auto corr =", Average(auto_corr), ", sd auto corr =", np.std(auto_corr))
print("average trx seller =", Average(trx_order_seller), ", sd trx_order_seller =", np.std(trx_order_seller))
print("average trx buyer =", Average(trx_order_buyer), ", sd trx_order_buyer =", np.std(trx_order_buyer))
print("average q =", Average(q))

save = -1
while (save != 0) & (save != 1):
    save = int(input('Please enter ''1'' to save , ''0'' otherwise:  '))

if save == 1:
    dict = {
        "auto_corr": auto_corr,
        "trx_order_seller": trx_order_seller,
        "trx_order_buyer": trx_order_buyer
    }
    df = pd.DataFrame(dict)
    df.to_stata("simulation_output_large_2.dta")
    


Please enter 1 for constrained simulation , 0 for unconstrained:  1
Please wait......
The simulation is done
average auto corr = -0.4948706936916055
average trx seller = 0.7138560310225015
average trx buyer = -0.7057368910349534
average q = 127.352
Please enter 1 to save , 0 otherwise:  1


In [33]:
random.seed(1234)
np.random.seed(1234)
np.random.rand()

0.1915194503788923

In [34]:
# market 4
numberTraders = 20
maxCost = 149
Quantity = 3

# Run simulation with 200 iteration
iterations = 2000

# Create vector holding all the values and costs
Values = [[209, 198, 178],
[208, 197, 177],
[200, 180, 160],
[210, 199, 179],
[207, 196, 176],
[210, 190, 170],
[209, 189, 169],
[208, 188, 168],
[207, 187, 167],
[206, 186, 166],
[204, 184, 164],
[205, 185, 165],
[202, 191, 171],
[203, 183, 163],
[202, 182, 162],
[201, 181, 161],
[206, 195, 175],
[203, 192, 172],
[204, 193, 173],
[205, 194, 174]]
Costs = [[102, 122, 142],
[103, 123, 143],
[100, 120, 140],
[101, 121, 141],
[104, 124, 144],
[90, 110, 130],
[91, 111, 131],
[92, 112, 132],
[93, 113, 133],
[94, 114, 134],
[96, 116, 136],
[95, 115, 135],
[109, 129, 149],
[97, 117, 137],
[98, 118, 138],
[99, 119, 139],
[105, 125, 145],
[108, 128, 148],
[107, 127, 147],
[106, 126, 146]]


# Create vector holding autocorreltion of price change
# and correlation of transaction order and costs/values
auto_corr = []
trx_order_seller = []
trx_order_buyer = []
q = []
p = []
hist = []

# Create vector holding all buyers
buyers = []
for i in range(numberTraders):
    buyers.append(Buyer(Quantity))

# Create vector holding all sellers
sellers = []
for i in range(numberTraders):
    sellers.append(Seller(Quantity))

print("Please wait......")
for i in range(10000):
    # Initialize all buyers to not traded state and give them a reservation
    # price (random variable bounded above)
    valueVec = np.asarray(Values)

    for i in range(numberTraders):
        valueVec[i].sort()
        valueVec[i] = valueVec[i][::-1]
        buyers[i].Value = valueVec[i]
        buyers[i].Traded = 0
    
    # Initialize all sellers to not traded state and give them a reservation
    # cost (random variable bounded below)
    costVec = np.asarray(Costs)

    for i in range(numberTraders):
        costVec[i].sort()
        sellers[i].Cost = costVec[i]
        sellers[i].Traded = 0


    # Initialize the order book vector. Please see function description for the
    # values held in each index   
    orderBookValues = initializeBook(maxCost)

    # Initialize vector with transaction prices, values, and costs (update as iterations execute).
    transactionPrices = []

    # Initial quantity traded is 0
    #quantity = 0
    tradedValues = []
    tradedCosts  = []
    h = []

    for i in range(iterations):
        # Stop the loop if all buyers and sellers have already traded all their goods.
        count = 0
        for i in range(numberTraders):
            if buyers[i].Traded == Quantity:
                count += 1
        if count == numberTraders:
            break
        # Attempt a trade or a new bid/ask (report update if trade occurs). 
        orderBookValues = doTrade(buyers, sellers, orderBookValues, numberTraders, Quantity)
    
        # Record transaction price, update surplus and quantity, mark traders 
        # to record that they have traded, and reinitialize the order book
        # if a trade occured
        if orderBookValues[6] > 0:
            transactionPrices.append(orderBookValues[6])
            #surplus = surplus + orderBookValues[7]
            tradedValues.append(buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded])
            tradedCosts.append(sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded])
            h.append([sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded], 
                      buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded], 
                      orderBookValues[6]])
            buyers[orderBookValues[1]].Traded += 1
            sellers[orderBookValues[4]].Traded += 1
            orderBookValues = initializeBook(maxCost)
            
        
    # gen transaction order and price change
    order = []
    for i in range(1, len(transactionPrices) + 1):
        order.append(i)

    all_priceChange = []
    for i in range(1, len(transactionPrices)):
        all_priceChange.append(transactionPrices[i] - transactionPrices[i - 1])

    priceChange = []
    lag_priceChange = []
    for i in range(1, len(all_priceChange)):
        priceChange.append(all_priceChange[i])
        lag_priceChange.append(all_priceChange[i-1])


    # Calculate correlation
    price_auto_corr, _ = pearsonr(priceChange, lag_priceChange)
    seller_corr, _ = pearsonr(Rank(tradedCosts), order)
    buyer_corr, _ = pearsonr(Rank(tradedValues), order)

    auto_corr.append(price_auto_corr)
    trx_order_seller.append(seller_corr)
    trx_order_buyer.append(buyer_corr)
    q.append(len(order))
    hist.append(h)

print("The simulation is done")

print("average auto corr =", Average(auto_corr), ", sd auto corr =", np.std(auto_corr))
print("average trx seller =", Average(trx_order_seller), ", sd trx_order_seller =", np.std(trx_order_seller))
print("average trx buyer =", Average(trx_order_buyer), ", sd trx_order_buyer =", np.std(trx_order_buyer))
print("average q =", Average(q))

save = -1
while (save != 0) & (save != 1):
    save = int(input('Please enter ''1'' to save , ''0'' otherwise:  '))

if save == 1:
    dict = {
        "auto_corr": auto_corr,
        "trx_order_seller": trx_order_seller,
        "trx_order_buyer": trx_order_buyer
    }
    df = pd.DataFrame(dict)
    df.to_stata("simulation_output_large_4.dta")

Please wait......
The simulation is done
average auto corr = -0.48883021506084634
average trx seller = 0.7116980105584882
average trx buyer = -0.6826662902298771
average q = 60.0
Please enter 1 to save , 0 otherwise:  1


In [19]:
random.seed(1234)
np.random.seed(1234)
np.random.rand()

0.1915194503788923

In [20]:
# market 5
numberTraders = 50
maxCost = 120
Quantity = 3

# Run simulation with 200 iteration
iterations = 2000

# Create vector holding all the values and costs
Values = [[140, 139, 138],
[139, 138, 137],
[142, 141, 140],
[141, 140, 139],
[138, 137, 136],
[123, 122, 121],
[125, 124, 123],
[124, 123, 122],
[127, 126, 125],
[126, 125, 124],
[108, 107, 106],
[111, 110, 109],
[112, 111, 110],
[109, 108, 107],
[110, 109, 108],
[150, 149, 100],
[149, 148, 147],
[133, 132, 131],
[136, 135, 134],
[137, 136, 135],
[134, 133, 132],
[135, 134, 133],
[122, 121, 120],
[121, 120, 119],
[120, 119, 118],
[119, 118, 117],
[118, 117, 116],
[103, 102, 101],
[107, 106, 105],
[106, 105, 104],
[105, 104, 103],
[104, 103, 102],
[147, 146, 145],
[146, 145, 144],
[145, 144, 143],
[144, 143, 142],
[143, 142, 141],
[128, 127, 126],
[129, 128, 127],
[130, 129, 128],
[131, 130, 129],
[132, 131, 130],
[150, 101, 100],
[150, 149, 148],
[148, 147, 146],
[114, 113, 112],
[115, 114, 113],
[116, 115, 114],
[117, 116, 115],
[113, 112, 111]]
Costs = [[62, 91, 112],
[63, 92, 113],
[60, 89, 110],
[61, 90, 111],
[64, 93, 114],
[50, 79, 100],
[52, 81, 102],
[54, 83, 104],
[58, 79, 108],
[56, 77, 106],
[57, 78, 107],
[54, 75, 104],
[55, 76, 105],
[52, 73, 94],
[70, 91, 120],
[69, 90, 119],
[51, 72, 93],
[50, 71, 92],
[69, 98, 119],
[66, 95, 116],
[65, 94, 115],
[68, 97, 118],
[67, 96, 117],
[59, 80, 109],
[60, 81, 110],
[61, 82, 111],
[62, 83, 112],
[63, 84, 113],
[51, 80, 101],
[53, 82, 103],
[57, 78, 99],
[53, 74, 95],
[54, 75, 96],
[55, 76, 97],
[56, 77, 98],
[55, 84, 105],
[56, 85, 106],
[57, 86, 107],
[58, 87, 108],
[59, 88, 109],
[53, 74, 103],
[52, 73, 102],
[51, 72, 101],
[50, 71, 100],
[70, 99, 120],
[67, 88, 117],
[66, 87, 116],
[65, 86, 115],
[64, 85, 114],
[68, 89, 118]]


# Create vector holding autocorreltion of price change
# and correlation of transaction order and costs/values
auto_corr = []
trx_order_seller = []
trx_order_buyer = []
q = []
p = []
hist = []

# Create vector holding all buyers
buyers = []
for i in range(numberTraders):
    buyers.append(Buyer(Quantity))

# Create vector holding all sellers
sellers = []
for i in range(numberTraders):
    sellers.append(Seller(Quantity))

print("Please wait......")
for i in range(10000):
    # Initialize all buyers to not traded state and give them a reservation
    # price (random variable bounded above)
    valueVec = np.asarray(Values)

    for i in range(numberTraders):
        valueVec[i].sort()
        valueVec[i] = valueVec[i][::-1]
        buyers[i].Value = valueVec[i]
        buyers[i].Traded = 0
    
    # Initialize all sellers to not traded state and give them a reservation
    # cost (random variable bounded below)
    costVec = np.asarray(Costs)

    for i in range(numberTraders):
        costVec[i].sort()
        sellers[i].Cost = costVec[i]
        sellers[i].Traded = 0


    # Initialize the order book vector. Please see function description for the
    # values held in each index   
    orderBookValues = initializeBook(maxCost)

    # Initialize vector with transaction prices, values, and costs (update as iterations execute).
    transactionPrices = []

    # Initial quantity traded is 0
    #quantity = 0
    tradedValues = []
    tradedCosts  = []
    h = []

    for i in range(iterations):
        # Stop the loop if all buyers and sellers have already traded all their goods.
        count = 0
        for i in range(numberTraders):
            if buyers[i].Traded == Quantity:
                count += 1
        if count == numberTraders:
            break
        # Attempt a trade or a new bid/ask (report update if trade occurs). 
        orderBookValues = doTrade(buyers, sellers, orderBookValues, numberTraders, Quantity)
    
        # Record transaction price, update surplus and quantity, mark traders 
        # to record that they have traded, and reinitialize the order book
        # if a trade occured
        if orderBookValues[6] > 0:
            transactionPrices.append(orderBookValues[6])
            #surplus = surplus + orderBookValues[7]
            tradedValues.append(buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded])
            tradedCosts.append(sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded])
            h.append([sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded], 
                      buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded], 
                      orderBookValues[6]])
            buyers[orderBookValues[1]].Traded += 1
            sellers[orderBookValues[4]].Traded += 1
            orderBookValues = initializeBook(maxCost)
            
        
    # gen transaction order and price change
    order = []
    for i in range(1, len(transactionPrices) + 1):
        order.append(i)

    all_priceChange = []
    for i in range(1, len(transactionPrices)):
        all_priceChange.append(transactionPrices[i] - transactionPrices[i - 1])

    priceChange = []
    lag_priceChange = []
    for i in range(1, len(all_priceChange)):
        priceChange.append(all_priceChange[i])
        lag_priceChange.append(all_priceChange[i-1])


    # Calculate correlation
    price_auto_corr, _ = pearsonr(priceChange, lag_priceChange)
    seller_corr, _ = pearsonr(Rank(tradedCosts), order)
    buyer_corr, _ = pearsonr(Rank(tradedValues), order)

    auto_corr.append(price_auto_corr)
    trx_order_seller.append(seller_corr)
    trx_order_buyer.append(buyer_corr)
    q.append(len(order))
    hist.append(h)

print("The simulation is done")

print("average auto corr =", Average(auto_corr), ", sd auto corr =", np.std(auto_corr))
print("average trx seller =", Average(trx_order_seller), ", sd trx_order_seller =", np.std(trx_order_seller))
print("average trx buyer =", Average(trx_order_buyer), ", sd trx_order_buyer =", np.std(trx_order_buyer))
print("average q =", Average(q))

save = -1
while (save != 0) & (save != 1):
    save = int(input('Please enter ''1'' to save , ''0'' otherwise:  '))

if save == 1:
    dict = {
        "auto_corr": auto_corr,
        "trx_order_seller": trx_order_seller,
        "trx_order_buyer": trx_order_buyer
    }
    df = pd.DataFrame(dict)
    df.to_stata("simulation_output_large_5.dta")

Please wait......
The simulation is done
average auto corr = -0.4955771465002771
average trx seller = 0.5744839442047573
average trx buyer = -0.15533210793884006
average q = 107.1548
Please enter 1 to save , 0 otherwise:  1


In [22]:
class Seller:
    # This class defines a seller. A seller has a reservation cost below
    # which he will not trade. Since each seller can only trade once, we
    # record whether a seller traded or not. A seller generates a bid price
    # (which varies depending if the simulation runs with or without
    # constraint)
    
    def __init__(self, quantity):
        self.Cost = []
        self.Traded = 0
        self.Quantity = quantity
    
    # Generate ask offer
    def formAskPrice(self):
        potentialAsk = self.Cost[self.Traded] + ((np.random.rand() * np.absolute(1000 - self.Cost[self.Traded])))
        
        return potentialAsk

In [23]:
random.seed(1234)
np.random.seed(1234)
np.random.rand()

0.1915194503788923

In [24]:
# market 3
numberTraders = 20
maxCost = 120
Quantity = 3

# Run simulation with 200 iteration
iterations = 2000

# Create vector holding all the values and costs
Values = [[808, 488, 168],
[792, 472, 152],
[840, 520, 200],
[824, 504, 184],
[776, 456, 136],
[1000, 680, 360],
[984, 664, 344],
[968, 648, 328],
[952, 632, 312],
[936, 616, 296],
[904, 584, 264],
[920, 600, 280],
[696, 376, 56],
[888, 568, 248],
[872, 552, 232],
[856, 536, 216],
[760, 440, 120],
[712, 392, 72],
[728, 408, 88],
[744, 424, 104]]
Costs = [[192, 512, 832],
[208, 528, 848],
[160, 480, 800],
[176, 496, 816],
[224, 544, 864],
[0, 320, 640],
[16, 336, 656],
[32, 352, 672],
[48, 368, 688],
[64, 384, 704],
[96, 416, 736],
[80, 400, 720],
[304, 624, 944],
[112, 432, 752],
[128, 448, 768],
[144, 464, 784],
[240, 560, 880],
[288, 608, 928],
[272, 592, 912],
[256, 576, 896]]


# Create vector holding autocorreltion of price change
# and correlation of transaction order and costs/values
auto_corr = []
trx_order_seller = []
trx_order_buyer = []
q = []
p = []
hist = []

# Create vector holding all buyers
buyers = []
for i in range(numberTraders):
    buyers.append(Buyer(Quantity))

# Create vector holding all sellers
sellers = []
for i in range(numberTraders):
    sellers.append(Seller(Quantity))

print("Please wait......")
for i in range(10000):
    # Initialize all buyers to not traded state and give them a reservation
    # price (random variable bounded above)
    valueVec = np.asarray(Values)

    for i in range(numberTraders):
        valueVec[i].sort()
        valueVec[i] = valueVec[i][::-1]
        buyers[i].Value = valueVec[i]
        buyers[i].Traded = 0
    
    # Initialize all sellers to not traded state and give them a reservation
    # cost (random variable bounded below)
    costVec = np.asarray(Costs)

    for i in range(numberTraders):
        costVec[i].sort()
        sellers[i].Cost = costVec[i]
        sellers[i].Traded = 0


    # Initialize the order book vector. Please see function description for the
    # values held in each index   
    orderBookValues = initializeBook(maxCost)

    # Initialize vector with transaction prices, values, and costs (update as iterations execute).
    transactionPrices = []

    # Initial quantity traded is 0
    #quantity = 0
    tradedValues = []
    tradedCosts  = []
    h = []

    for i in range(iterations):
        # Stop the loop if all buyers and sellers have already traded all their goods.
        count = 0
        for i in range(numberTraders):
            if buyers[i].Traded == Quantity:
                count += 1
        if count == numberTraders:
            break
        # Attempt a trade or a new bid/ask (report update if trade occurs). 
        orderBookValues = doTrade(buyers, sellers, orderBookValues, numberTraders, Quantity)
    
        # Record transaction price, update surplus and quantity, mark traders 
        # to record that they have traded, and reinitialize the order book
        # if a trade occured
        if orderBookValues[6] > 0:
            transactionPrices.append(orderBookValues[6])
            #surplus = surplus + orderBookValues[7]
            tradedValues.append(buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded])
            tradedCosts.append(sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded])
            h.append([sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded], 
                      buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded], 
                      orderBookValues[6]])
            buyers[orderBookValues[1]].Traded += 1
            sellers[orderBookValues[4]].Traded += 1
            orderBookValues = initializeBook(maxCost)
            
        
    # gen transaction order and price change
    order = []
    for i in range(1, len(transactionPrices) + 1):
        order.append(i)

    all_priceChange = []
    for i in range(1, len(transactionPrices)):
        all_priceChange.append(transactionPrices[i] - transactionPrices[i - 1])

    priceChange = []
    lag_priceChange = []
    for i in range(1, len(all_priceChange)):
        priceChange.append(all_priceChange[i])
        lag_priceChange.append(all_priceChange[i-1])


    # Calculate correlation
    price_auto_corr, _ = pearsonr(priceChange, lag_priceChange)
    seller_corr, _ = pearsonr(Rank(tradedCosts), order)
    buyer_corr, _ = pearsonr(Rank(tradedValues), order)

    auto_corr.append(price_auto_corr)
    trx_order_seller.append(seller_corr)
    trx_order_buyer.append(buyer_corr)
    q.append(len(order))
    hist.append(h)

print("The simulation is done")

print("average auto corr =", Average(auto_corr), ", sd auto corr =", np.std(auto_corr))
print("average trx seller =", Average(trx_order_seller), ", sd trx_order_seller =", np.std(trx_order_seller))
print("average trx buyer =", Average(trx_order_buyer), ", sd trx_order_buyer =", np.std(trx_order_buyer))
print("average q =", Average(q))

save = -1
while (save != 0) & (save != 1):
    save = int(input('Please enter ''1'' to save , ''0'' otherwise:  '))

if save == 1:
    dict = {
        "auto_corr": auto_corr,
        "trx_order_seller": trx_order_seller,
        "trx_order_buyer": trx_order_buyer
    }
    df = pd.DataFrame(dict)
    df.to_stata("simulation_output_large_3.dta")
    


Please wait......
The simulation is done
average auto corr = -0.4831446684205202
average trx seller = 0.49899726812818135
average trx buyer = -0.572561206460607
average q = 33.6766
Please enter 1 to save , 0 otherwise:  1


In [11]:
times = []  # will record number of transactions for every experiment
header = ['cost', 'value', 'price']

for i in hist:
    df = pd.DataFrame(i, columns=header)
    df['seller'] = df['cost'].rank()  # calculate seller rank by costs
    df['buyer'] = df['value'].rank()  # calculate buyer rank by values
    df['time'] = df.index+1  # transaction order is index+1
    times.append(len(df))

In [15]:
# An example of each experiment
df

,cost,value,price,seller,buyer,time
0,55,145,76.678281,6.0,46.5,1
1,53,146,84.304435,4.0,48.5,2
2,63,148,141.154334,14.0,52.5,3
3,54,149,125.965123,5.0,54.5,4
4,57,128,83.514498,8.0,25.0,5
5,52,143,120.674751,3.0,42.5,6
6,50,129,125.997321,1.0,26.0,7
7,56,142,59.049307,7.0,40.5,8
8,65,136,129.511240,16.0,33.0,9
9,60,145,118.035692,11.0,46.5,10
